In [23]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/patient_info2/observations.csv"

data = spark.read.format("csv").option("inferSchema", True).option("header", False).load(bucket_file)
data.createOrReplaceTempView("data")
data = spark.sql("""
select _c0 as patient,
_c1 as height, _c2 as weight, _c3 as BMI, _c4 as cholesterol,
_c5 as sodium, _c6 as glucose
from data""")

data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------+-----+-----------+------+-------+
|             patient|height|weight|  BMI|cholesterol|sodium|glucose|
+--------------------+------+------+-----+-----------+------+-------+
|00c343e4-e4eb-4c6...|176.61|116.12|37.23|      163.0| 141.0|   83.0|
|00eec485-f89d-472...| 172.3| 94.04|31.68|      207.0| 140.0|  116.0|
|017299f2-de8f-43a...|177.38|135.75|43.15|      161.0| 139.0|   72.0|
|018dd53e-4b90-41c...|163.33|136.74|51.26|      162.0| 143.0|   71.0|
|0190ce14-8897-495...| 161.1| 72.92| 28.1|      196.0| 137.0|   75.0|
|01e7a88d-34ac-494...|173.35|105.97|35.27|      169.0| 143.0|   85.0|
|01f649ca-232e-4c1...|151.11|102.05|44.69|      199.0| 140.0|   94.0|
|0216c768-a4c0-46a...|169.24| 96.32|33.63|      197.0| 140.0|   64.0|
|023230b8-01ca-4d3...|157.84| 92.49|37.13|      179.0| 142.0|   88.0|
|02357156-a39f-45f...|185.33|109.37|31.84|      173.0| 143.0|   74.0|
|0259aa6f-d531-43f...|163.33|100.15|37.54|      233.0| 140.0|  100.0|
|029f2ed6-60a0-4d4..

In [24]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="patient", outputCol="ID")
indexed = indexer.fit(data).transform(data)

indexed = indexed.drop("patient")
indexed.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+-----------+------+-------+-------+
|height|weight|  BMI|cholesterol|sodium|glucose|     ID|
+------+------+-----+-----------+------+-------+-------+
|176.61|116.12|37.23|      163.0| 141.0|   83.0|21687.0|
| 172.3| 94.04|31.68|      207.0| 140.0|  116.0|12249.0|
|177.38|135.75|43.15|      161.0| 139.0|   72.0|28222.0|
|163.33|136.74|51.26|      162.0| 143.0|   71.0| 5018.0|
| 161.1| 72.92| 28.1|      196.0| 137.0|   75.0|23116.0|
|173.35|105.97|35.27|      169.0| 143.0|   85.0|10539.0|
|151.11|102.05|44.69|      199.0| 140.0|   94.0|13891.0|
|169.24| 96.32|33.63|      197.0| 140.0|   64.0| 4396.0|
|157.84| 92.49|37.13|      179.0| 142.0|   88.0|24275.0|
|185.33|109.37|31.84|      173.0| 143.0|   74.0|21453.0|
|163.33|100.15|37.54|      233.0| 140.0|  100.0|17018.0|
|170.46|111.64|38.42|      171.0| 144.0|   90.0|29378.0|
|170.45|112.29|38.65|      199.0| 144.0|   78.0| 9794.0|
|166.01|  92.5|33.56|      178.0| 143.0|   72.0|30135.0|
|175.46| 92.34|29.99|      160.

In [25]:
# creating a vector of features for each row
from pyspark.ml.feature import VectorAssembler

features = ('ID', 'height', 'weight', 'BMI', 'cholesterol', 'sodium', 'glucose')

assembler = VectorAssembler(inputCols=features, outputCol = "features")

raw_data = assembler.transform(indexed)
raw_data.select("features").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+
|features                                       |
+-----------------------------------------------+
|[21687.0,176.61,116.12,37.23,163.0,141.0,83.0] |
|[12249.0,172.3,94.04,31.68,207.0,140.0,116.0]  |
|[28222.0,177.38,135.75,43.15,161.0,139.0,72.0] |
|[5018.0,163.33,136.74,51.26,162.0,143.0,71.0]  |
|[23116.0,161.1,72.92,28.1,196.0,137.0,75.0]    |
|[10539.0,173.35,105.97,35.27,169.0,143.0,85.0] |
|[13891.0,151.11,102.05,44.69,199.0,140.0,94.0] |
|[4396.0,169.24,96.32,33.63,197.0,140.0,64.0]   |
|[24275.0,157.84,92.49,37.13,179.0,142.0,88.0]  |
|[21453.0,185.33,109.37,31.84,173.0,143.0,74.0] |
|[17018.0,163.33,100.15,37.54,233.0,140.0,100.0]|
|[29378.0,170.46,111.64,38.42,171.0,144.0,90.0] |
|[9794.0,170.45,112.29,38.65,199.0,144.0,78.0]  |
|[30135.0,166.01,92.5,33.56,178.0,143.0,72.0]   |
|[29460.0,175.46,92.34,29.99,160.0,141.0,98.0]  |
|[15720.0,173.81,108.56,35.93,172.0,139.0,71.0] |
|[23009.0,162.92,102.96,38.79,200.0,143.0,77.0] |


In [30]:
# scaling the feature vectors and creating a column with the output
from pyspark.ml.feature import StandardScaler

standardscaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")

raw_data = standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("scaled_features").show(5, truncate = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     scaled_features|
+--------------------+
|[2.47080205020966...|
|[1.39552977880842...|
|[3.21533524512460...|
|[0.57170123520782...|
|[2.63360816123238...|
+--------------------+
only showing top 5 rows

In [11]:
# training the k-means model using the scaled features column
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol = "scaled_features").setK(5).setSeed(1)
model = kmeans.fit(raw_data)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# performing the k-means clustering job
predictions = model.transform(raw_data)

In [33]:
# listing the number of data points in each cluster
clusters = predictions.select("prediction")
clusters.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8296|
|         3|             2600|
|         4|             5203|
|         2|             6372|
|         0|             7934|
+----------+-----------------+

In [34]:
# performing a heirarchical clustering job

from pyspark.ml.clustering import BisectingKMeans

# training the model
bkm = BisectingKMeans(featuresCol="scaled_features").setK(5).setSeed(1)
model = bkm.fit(raw_data)

# creating predictions 
predictions_b = model.transform(raw_data)

# listing the number of points in each cluster
cm = predictions_b.select("prediction")
cm.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8430|
|         3|             5189|
|         4|             3010|
|         2|             4951|
|         0|             8825|
+----------+-----------------+

In [35]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Make predictions


#Evaluate using Silhouette score
evaluator = ClusteringEvaluator()

silhouette_k = evaluator.evaluate(predictions)
silhouette_h = evaluator.evaluate(predictions_b)
print(f"""K-means silhouette with squared euclidean distance = {str(silhouette_k)} \n 
    Hierarchical silhouette with squared euclidean distance = {str(silhouette_h)}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

K-means silhouette with squared euclidean distance = -0.131598264118536 
 
        Hierarchical silhouette with squared euclidean distance = -0.23110782587064593